In [1]:
import cudf
import dask_cudf as dd
import xgboost as xgb

from dask_cuda import LocalCUDACluster
from dask.distributed import Client

In [2]:
cluster = LocalCUDACluster(
    n_workers=4,
    protocol="ucx",
    jit_unspill=True,
    # rmm_pool_size="28GiB",
    device_memory_limit="30GB",
)
client = Client(cluster)

/raid/cjarrett/mambaforge/envs/xgboost-old/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 33311 instead
  warnings.warn(
2023-02-27 10:26:28,137 - distributed.comm.ucx - WARNING - A CUDA context for device 0 (b'GPU-67be321d-9012-d150-fdc9-402004163697') already exists on process ID 64825. This is often the result of a CUDA-enabled library calling a CUDA runtime function before Dask-CUDA can spawn worker processes. Please make sure any such function calls don't happen at import time or in the global scope of a program.
2023-02-27 10:26:29,709 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2023-02-27 10:26:29,709 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space-10224/worker-cwbq391w', purging
2023-02-27 10:26:29,709 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
202

In [3]:
ddf = dd.read_parquet("/datasets/criteo/crit_orig_pq_1day", split_row_groups=4)
ddf = ddf.partitions[0]
ddf = ddf.fillna(0)

for n in range(1,27):
    col = "C" + str(n)
    ddf[col] = ddf[col].astype("category")

In [5]:
X, y = ddf.drop("label", axis=1), ddf["label"]
dtrain = xgb.dask.DaskDMatrix(client, X, y, enable_categorical=True)

2023-02-27 10:29:40,895 - distributed.worker - WARNING - RMM allocation of 14.15 MiB failed, spill-on-demand couldn't find any device memory to spill.
<ProxyManager dev_limit=27.94 GiB host_limit=377.64 GiB disk=0 B(0) host=575.39 MiB(1) dev=0 B(0)>, traceback:
  File "/raid/cjarrett/mambaforge/envs/xgboost-old/lib/python3.10/threading.py", line 973, in _bootstrap
    self._bootstrap_inner()
  File "/raid/cjarrett/mambaforge/envs/xgboost-old/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/raid/cjarrett/mambaforge/envs/xgboost-old/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/raid/cjarrett/mambaforge/envs/xgboost-old/lib/python3.10/site-packages/distributed/threadpoolexecutor.py", line 57, in _worker
    task.run()
  File "/raid/cjarrett/mambaforge/envs/xgboost-old/lib/python3.10/site-packages/distributed/_concurrent_futures_thread.py", line 65, in run
    result = self.fn(*self.args, **self.kwa

AssertionError: error

In [4]:
output = xgb.dask.train(
        client,
        {"verbosity": 2,
         "tree_method": "gpu_hist",
         "objective": "binary:logistic",
         "enable_categorical": True,
         "max_cat_to_one_hot": 2},
        dtrain,
        num_boost_round=4,
        # evals=[(dtrain, "train")],
    )

NameError: name 'dtrain' is not defined

In [15]:
prediction = xgb.dask.predict(client, output, dtrain)

2023-02-22 17:58:31,759 - distributed.worker - WARNING - Compute Failed
Key:       dispatched_predict-ce569f2da414c2cc095ece0d9abf9e32
Function:  dispatched_predict
args:      (<xgboost.core.Booster object at 0x7f94640a10c0>, {'data': <dask_cuda.proxy_object.ProxyObject at 0x7f9436fe3e50 of cudf.core.dataframe.DataFrame at 0x7f9436e45b40>, 'label': <dask_cuda.proxy_object.ProxyObject at 0x7f94640a0d30 of cudf.core.series.Series at 0x7f9436f931f0>})
kwargs:    {}
Exception: "ValueError('DataFrame.dtypes for data must be int, float, bool or category. When categorical type is supplied, The experimental DMatrix parameter`enable_categorical` must be set to `True`.  Invalid columns:C1: category, C2: category, C3: category, C4: category, C5: category, C6: category, C7: category, C8: category, C9: category, C10: category, C11: category, C12: category, C13: category, C14: category, C15: category, C16: category, C17: category, C18: category, C19: category, C20: category, C21: category, C22: cate

In [13]:
clf = xgb.dask.DaskXGBClassifier(
    tree_method="gpu_hist",
    enable_categorical=True,
    max_cat_to_onehot=1,
    learning_rate=0.3,
    max_depth=10,
    objective="binary:logistic",
    subsample=0.7,
)
clf.client = client
clf.fit(X,y)

/raid/cjarrett/mambaforge/envs/xgboost-old/lib/python3.10/site-packages/xgboost/dask.py:856: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
INFO:distributed.scheduler:Receive client connection: Client-worker-1615b936-b31c-11ed-a40d-5cff35c18da6
INFO:distributed.core:Starting established connection to ucx://127.0.0.1:56047
[17:47:53] task [xgboost.dask-3]:ucx://127.0.0.1:42855 got new rank 0
2023-02-22 17:47:55,680 - distributed.worker - WARNING - Compute Failed
Key:       dispatched_train-a2da436a-ef39-4229-86f9-884a831292ce
Function:  dispatched_train
args:      ({'base_score': None, 'booster': None, 'colsample_bylevel': None, 'colsample_bynode': None, 'colsample_bytree': None, 'eval_metric': None, 'gamma': None, 'gpu_id': None, 'grow_policy': None, 'interaction_constraints': None, 'learning_rate': 0.3, 'max_bin': None, 'max_cat_threshold': None, 'max_cat_to_onehot': 1, 'max_delta_step': None, 'max_depth': 10, 'max_leaves': 

XGBoostError: [17:47:55] /home/conda/feedstock_root/build_artifacts/xgboost-split_1667849645640/work/src/tree/updater_gpu_hist.cu:784: Exception in gpu_hist: [17:47:55] /home/conda/feedstock_root/build_artifacts/xgboost-split_1667849645640/work/src/collective/../common/common.h:46: /home/conda/feedstock_root/build_artifacts/xgboost-split_1667849645640/work/src/tree/gpu_hist/row_partitioner.cuh: 295: cudaErrorIllegalAddress: an illegal memory access was encountered
Stack trace:
  [bt] (0) /raid/cjarrett/mambaforge/envs/xgboost-old/lib/libxgboost.so(+0x20a75f) [0x7f4e7105275f]
  [bt] (1) /raid/cjarrett/mambaforge/envs/xgboost-old/lib/libxgboost.so(dh::ThrowOnCudaError(cudaError, char const*, int)+0x503) [0x7f4e713b5a33]
  [bt] (2) /raid/cjarrett/mambaforge/envs/xgboost-old/lib/libxgboost.so(void xgboost::tree::RowPartitioner::UpdatePositionBatch<__nv_dl_wrapper_t<__nv_dl_tag<void (xgboost::tree::GPUHistMakerDevice<xgboost::detail::GradientPairInternal<double> >::*)(std::vector<xgboost::t

In [ ]:
# Get a graph
graph = xgb.to_graphviz(clf, num_trees=1)
# Or get a matplotlib axis
ax = xgb.plot_tree(clf, num_trees=1)
# Get feature importances
clf.feature_importances_